In [147]:
import numpy as np
import pandas as pd
import re,math
import concurrent.futures
import multiprocessing
import threading
import time


data1 = pd.read_csv(r'pcc_cpu_test_multi_pcbi.csv').set_index(["Unnamed: 0"])
data2 = pd.read_excel(r'583drugs39cellsynergy.xlsx')
drug_names = list(set(list(data1["target"])))

In [148]:
data1

,target_neighbor,target,similarity,neighbors,alp,pcc,rmse
Unnamed: 0,,,,,,,
0,"'TOPOTECAN', 'SORAFENIB'","'MK-5108', 'SORAFENIB'",0.6391,43,1.0,0.795369,0.695401
1,"'MRK-003', 'SORAFENIB'","'MK-5108', 'SORAFENIB'",0.6418,43,1.0,0.795369,0.695401
2,"'PD325901', 'SORAFENIB'","'MK-5108', 'SORAFENIB'",0.6429,43,1.0,0.795369,0.695401
3,"'BEZ-235', 'SORAFENIB'","'MK-5108', 'SORAFENIB'",0.6446,43,1.0,0.795369,0.695401
4,"'VINORELBINE', 'MK-5108'","'MK-5108', 'SORAFENIB'",0.6454,43,1.0,0.795369,0.695401
...,...,...,...,...,...,...,...
22453,"'AZD1775', 'OXALIPLATIN'","'OXALIPLATIN', 'DINACICLIB'",0.7203,45,1.0,0.445882,0.906365
22454,"'OXALIPLATIN', 'MK-8776'","'OXALIPLATIN', 'DINACICLIB'",0.7290,45,1.0,0.445882,0.906365
22455,"'DASATINIB', 'OXALIPLATIN'","'OXALIPLATIN', 'DINACICLIB'",0.7304,45,1.0,0.445882,0.906365


In [149]:
data2

,DrugA,DrugB,A2058,A2780,A375,A427,CAOV3,COLO320DM,EFM192B,ES2,...,T47D,UACC62,VCAP,ZR751,DLD1,PA1,LNCAP,OCUBM,UWB1289BRCA1,UWB1289
0,MK-5108,SORAFENIB,-9.509511,2.601305,15.161752,6.220745,-16.354738,5.362729,-7.079908,8.048158,...,6.626663,3.879985,-1.480834,-8.658922,13.001908,4.829214,-8.151099,-0.382005,6.540280,3.433143
1,VINORELBINE,SUNITINIB,-13.161741,-4.034887,11.058178,10.462220,-15.571398,16.327184,-1.719933,8.892855,...,-14.378149,-10.711458,-16.797392,-11.847725,15.299948,-13.078937,-0.282404,-30.439278,2.127934,-0.951918
2,SUNITINIB,MK-8776,26.422538,14.481629,29.508245,17.509092,17.730928,10.321708,0.829514,5.771627,...,13.085715,18.842203,12.005982,5.950419,6.767119,18.859616,-8.476897,8.410700,-23.005229,-10.386139
3,5-FU,DINACICLIB,4.331695,-8.161129,-5.415934,-7.747474,-14.347426,4.807550,0.312107,3.407185,...,-2.721986,3.682573,14.421994,-13.582390,-3.641267,-5.732865,-0.848044,6.785054,3.041715,-0.233191
4,SUNITINIB,MK-2206,42.282160,22.492502,12.958680,15.567437,10.478930,15.083221,1.591496,11.978018,...,14.309170,34.743261,21.156761,14.449726,26.446116,39.771885,6.011933,-3.293458,38.233789,26.500062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,CARBOPLATIN,MK-2206,1.353449,11.711991,20.229157,9.172272,46.600767,0.585604,-37.695889,-2.104280,...,-43.917390,9.335640,-4.833250,-15.416716,-4.415779,17.830863,-31.203323,-24.813710,10.979468,5.623035
579,5-FU,LAPATINIB,9.192138,8.011125,7.497787,-4.179454,-6.777528,3.209346,7.238193,5.344512,...,14.909283,9.119946,10.632558,14.676307,11.554627,14.159219,-16.376497,3.696229,19.005875,3.884721
580,ETOPOSIDE,AZD1775,15.445011,12.377177,19.244666,-5.013825,37.364847,8.589955,9.771696,3.769656,...,-20.698431,25.102521,20.279953,3.751568,-4.294381,-1.435133,21.451106,12.104305,7.938365,85.228585
581,BORTEZOMIB,MK-8669,-6.295089,4.790599,-23.326593,-2.704414,0.085578,-17.344643,19.191305,17.416089,...,-11.069717,-13.513875,9.045255,-0.305533,9.479898,-4.287754,7.589511,6.417144,4.800447,-15.072407


In [150]:
# 转换数值
columns_to_process = data2.columns[2:]
for column in columns_to_process:
    data2[column] = data2[column].apply(lambda x: 1 if x > 30 else -1)

In [151]:
data2

,DrugA,DrugB,A2058,A2780,A375,A427,CAOV3,COLO320DM,EFM192B,ES2,...,T47D,UACC62,VCAP,ZR751,DLD1,PA1,LNCAP,OCUBM,UWB1289BRCA1,UWB1289
0,MK-5108,SORAFENIB,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,VINORELBINE,SUNITINIB,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,SUNITINIB,MK-8776,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,5-FU,DINACICLIB,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,SUNITINIB,MK-2206,1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,-1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,CARBOPLATIN,MK-2206,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
579,5-FU,LAPATINIB,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
580,ETOPOSIDE,AZD1775,-1,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
581,BORTEZOMIB,MK-8669,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [152]:
# 组合index
data2['index'] = data2.apply(lambda row: f"'{row[0]}', '{row[1]}'", axis=1)
data2.set_index('index', inplace=True)
data2.drop(columns=[data2.columns[0], data2.columns[1]], inplace=True)

In [153]:
data2

,A2058,A2780,A375,A427,CAOV3,COLO320DM,EFM192B,ES2,HCT116,HT144,...,T47D,UACC62,VCAP,ZR751,DLD1,PA1,LNCAP,OCUBM,UWB1289BRCA1,UWB1289
index,,,,,,,,,,,,,,,,,,,,,
"'MK-5108', 'SORAFENIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'VINORELBINE', 'SUNITINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'SUNITINIB', 'MK-8776'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'DINACICLIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'SUNITINIB', 'MK-2206'",1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,1,-1,-1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"'CARBOPLATIN', 'MK-2206'",-1,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'LAPATINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'ETOPOSIDE', 'AZD1775'",-1,-1,-1,-1,1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [154]:
# 获取第三列和第五列的数值
t1 = data1.iloc[:, 2]
t2 = data1.iloc[:, 4]

# 计算指数函数值
exponential_values = np.exp(-((1 - t1) ** 2) / (2 * t2 ** 2))

# 将指数函数值添加为新列到data1中
data1['w'] = exponential_values

In [155]:
# 分组并获取对应的索引行
grouped = data1.groupby('target')
result = pd.DataFrame()

pps_list=[]
for group_name, group_data in grouped:
    # 邻居名字
    target_neighbor = group_data['target_neighbor'] 
    # 权重
    ws = group_data['w']
    # 目标标签
    # list(group_data['target'])[0]
    rs = data2.loc[group_name]
    # 邻居标签们
    s_list = []
    for i in target_neighbor:
        s_list.append(list(data2.loc[i]))
    # 分子未求和
    fz = []
    for i,j in zip(ws,s_list):
        fz.append([i * val for val in j])
    # 求和后的分子
    tgfz = np.array(fz[0])
    for i in fz[1:]:
        tgfz += np.array(i)
    # 权重和
    total_ws = sum(ws)
    # 预测标签
    ps = [ val/total_ws for val in tgfz]
    pps = []
    for i in ps:
        if i<0:
            pps.append(-1)
        else:
            pps.append(1)
    pps_list.append(pps)

In [156]:
predict_data2 = pd.concat([pd.DataFrame(sublist).T for sublist in pps_list], ignore_index=True)
predict_data2

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
579,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
580,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
581,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [157]:
predict_data2 = predict_data2.set_index(data2.index)
predict_data2

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
index,,,,,,,,,,,,,,,,,,,,,
"'MK-5108', 'SORAFENIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'VINORELBINE', 'SUNITINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
"'SUNITINIB', 'MK-8776'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'DINACICLIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'SUNITINIB', 'MK-2206'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"'CARBOPLATIN', 'MK-2206'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'LAPATINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'ETOPOSIDE', 'AZD1775'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [158]:
predict_data2.columns = data2.columns
predict_data2

,A2058,A2780,A375,A427,CAOV3,COLO320DM,EFM192B,ES2,HCT116,HT144,...,T47D,UACC62,VCAP,ZR751,DLD1,PA1,LNCAP,OCUBM,UWB1289BRCA1,UWB1289
index,,,,,,,,,,,,,,,,,,,,,
"'MK-5108', 'SORAFENIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'VINORELBINE', 'SUNITINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
"'SUNITINIB', 'MK-8776'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'DINACICLIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'SUNITINIB', 'MK-2206'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"'CARBOPLATIN', 'MK-2206'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'5-FU', 'LAPATINIB'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
"'ETOPOSIDE', 'AZD1775'",-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [159]:
# 差异率
diff = predict_data2 != data2
diff_rate = diff.mean().mean()
diff_rate

0.0948673967541892